In [1]:
import os
import sys
import numpy as np

In [2]:
file_clicks_basename = 'yoochoose-clicks-0.01.dat'
file_buys_basename = 'yoochoose-buys-0.01.dat'
file_test_basename = 'yoochoose-test.dat'
file_what_to_buy_features_basename = 'wtb-0.01.csv'
file_buy_or_not_features_basename = 'bon-0.01.npy'
data_path = 'data'
cache_path = 'cache'
file_clicks = os.path.join(data_path, file_clicks_basename)
file_buys = os.path.join(data_path, file_buys_basename)
file_test = os.path.join(data_path, file_test_basename)
file_what_to_buy_features = os.path.join(cache_path, file_what_to_buy_features_basename)
file_buy_or_not_features = os.path.join(cache_path, file_buy_or_not_features_basename)

In [3]:
effective_columns_names = ['Session ID', 'Timestamp', 'Item ID']

In [4]:
from src.main.read_and_write_data import read_clicks

clicks = read_clicks(file_clicks, usecols=effective_columns_names)\
    .sort_values('Timestamp')\
    .reset_index(drop=True)

In [5]:
from src.main.preprocess_data import df_group_by_session_id

clicks_grouped_by_session_id, clicks_grouped_by_session_id_keys = df_group_by_session_id(clicks)

In [6]:
from src.main.read_and_write_data import features_from_csv, features_to_csv
from src.main.feature_extraction import extract_what_to_buy

if os.path.isfile(file_what_to_buy_features):
        what_to_buy = features_from_csv(file_what_to_buy_features, ['Session ID', 'Item ID'],
                                        {'F3': 'Counts', 'F6': 'Sequent Clicks', 'F7': 'Time Difference'})
else:
    what_to_buy = extract_what_to_buy(clicks_grouped_by_session_id)
    features_to_csv(file_what_to_buy_features, what_to_buy.values())

In [7]:
from src.main.feature_extraction import extract_buy_or_not

if os.path.isfile(file_buy_or_not_features):
        buy_or_not = np.load(file_buy_or_not_features)
else:
    buy_or_not = extract_buy_or_not(clicks_grouped_by_session_id, what_to_buy)
    np.save(file_buy_or_not_features, buy_or_not)

Extracting feature: P2
Extraction completed in 0 minutes 0 seconds
Extracting feature: P3
Extraction completed in 0 minutes 8 seconds
Extracting feature: P1
Extraction completed in 0 minutes 0 seconds
Extracting feature: P6
Extraction completed in 0 minutes 0 seconds
Extracting feature: P4
Extraction completed in 0 minutes 30 seconds
Extracting feature: P10
Extraction completed in 0 minutes 0 seconds
Extracting feature: P11
Extraction completed in 0 minutes 42 seconds
Extracting feature: P5
Extraction completed in 0 minutes 0 seconds


In [8]:
from src.main.read_and_write_data import read_buys
from src.main.feature_extraction import extract_buys

buys = read_buys(file_buys, usecols=effective_columns_names)
_, buys_grouped_by_session_id_keys = df_group_by_session_id(buys)
buys_result = extract_buys(clicks_grouped_by_session_id_keys, buys_grouped_by_session_id_keys)

In [9]:
for feature_index in xrange(buy_or_not.shape[1]):
    print np.sort(buy_or_not[:, feature_index])

[   1.    1.    1. ...,   76.   76.  180.]
[  1.   1.   1. ...,  22.  23.  48.]
[        0.         0.         0. ...,  19360452.  19771408.  23952820.]
[       0.        0.        0. ...,  3595598.  3595847.  3596386.]
[  0.00000000e+00   0.00000000e+00   0.00000000e+00 ...,   3.40282347e+38
   3.40282347e+38   3.40282347e+38]
[  1.   1.   1. ...,  22.  23.  48.]
[  1.   1.   1. ...,  22.  23.  48.]
[  1.00000000e+00   3.00000000e+00   8.00000000e+00 ...,   2.14748365e+09
   2.14748365e+09   2.14748365e+09]


In [10]:
np.unique(buy_or_not[:, 4].flatten())

array([  0.00000000e+00,   5.00000000e-01,   1.00000000e+00, ...,
         3.59320700e+06,   3.59543000e+06,   3.40282347e+38], dtype=float32)